In [0]:
!pip install feature-engine

In [0]:
from feature_engine import imputation, selection

In [0]:
# SEMMA

## SAMPLE

df = spark.table("analytics.olist.abt_ativacao").toPandas()
df.head(2)

In [0]:
# Check safras
df['dtRef'].unique()

In [0]:
# create OOT using the last moment of the data reference
df_oot = df[df['dtRef'] == df['dtRef'].max()].copy()

# train dataset
df_train = df[df['dtRef'] < df['dtRef'].max()].copy()

In [0]:
from sklearn import model_selection

features = df_train.columns[4:].tolist()
target = 'flagChurn'

X = df_train[features]
y = df_train[target]

# Teain and test split
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                                                    random_state=42, 
                                                                    test_size=0.2,
                                                                    stratify=y)

print("train target distribution: ", y_train.mean())
print(" test target distribution: ", y_test.mean())

In [0]:
## EXPLORE

# Check NA - count
train_isna = X_train.isna().sum()
train_isna[train_isna > 0]

In [0]:
# Check NA - mean
train_isna = X_train.isna().mean()
train_isna[train_isna > 0]

In [0]:
# X_train['mediaAvaliacao'].describe()

In [0]:
# X_train['mediaAvaliacao28d'].describe()

In [0]:
## MODIFY

### STEP: Mean Imput

features_imput_mean = [
    'mediaCubagemProduto',
    'mediaAvaliacao168d',
    'mediaAvaliacao336d',
    'mediaAvaliacao',
    'mediaTamanhoNomeProduto',
    'mediaTamanhoDescricaoProduto',
    'mediaPesoGramas',
]

imputer_mean = imputation.MeanMedianImputer(imputation_method='mean', variables=features_imput_mean)


In [0]:
### STEP: Zero Imput

features_imput_zero = [
    'categoriasDistintas',
    'qtdMediaFotos',
    'pctProdutosComFotos',
    'pctProdutosComDescricao',
    'cliUfAC',
    'cliUfAL',
    'cliUfAM',
    'cliUfAP',
    'cliUfBA',
    'cliUfCE',
    'cliUfDF',
    'cliUfES',
    'cliUfGO',
    'cliUfMA',
    'cliUfMG',
    'cliUfMS',
    'cliUfMT',
    'cliUfPA',
    'cliUfPB',
    'cliUfPE',
    'cliUfPI',
    'cliUfPR',
    'cliUfRJ',
    'cliUfRN',
    'cliUfRO',
    'cliUfRR',
    'cliUfRS',
    'cliUfSC',
    'cliUfSE',
    'cliUfSP',
    'cliUfTO',
    'pctNota1',
    'pctNota2',
    'pctNota3',
    'pctNota4',
    'pctNota5',
    'pctNotaBaixa',
    'maiorCategoriaTotalPedidos',
    'shareTopCategorias',
    'pctautomotivo',
    'pctbeleza_saude',
    'pctbrinquedos',
    'pctcama_mesa_banho',
    'pctcool_stuff',
    'pcteletroportateis',
    'pctesporte_lazer',
    'pctferramentas_jardim',
    'pctinformatica_acessorios',
    'pctmoveis_decoracao',
    'pctmoveis_escritorio',
    'pctperfumaria',
    'pctrelogios_presentes',
    'pcttelefonia',
    'pctutilidades_domesticas',
    'pctPedidosEntreguesNoPrazo',
    'qtdCliente',
    'cliNovo',
    'cliPontual',
    'cliRecorrente',
    'qtdProdutosDistintos',
    'maiorCategoriaTotalReceita',
]

imputer_zero = imputation.ArbitraryNumberImputer(arbitrary_number=0, variables=features_imput_zero)


In [0]:
### STEP: One Imput

features_imput_one = [
    'pctTendencia1m_2m',
    'pctTendencia1m_3m',
    'pctTendencia1m_4m',
    'pctTendencia1m_5m',
    'pctTendencia1m_6m',
    'pctTendencia1m_12m',
    'mediaQtdeParcelas',
]

imputer_one = imputation.ArbitraryNumberImputer(arbitrary_number=1, variables=features_imput_one)


In [0]:
### STEP: Category Imput (missing category)

flag_missing = [
'maiorCategoriaVenda',
'maiorCategoriaQtdePedidos',
'vendedorRegiao',
]

imputer_flag_missing = imputation.CategoricalImputer(fill_value="MISSING", variables=flag_missing)


In [0]:
### STEP: Max Imput

features_imput_max = ['qtdeMediaDiasEntrePedidos']

imputer_max = imputation.EndTailImputer(imputation_method='max', tail='right', fold=1, variables=features_imput_max)


In [0]:
### STEP: Revoming variables

features_drop = [
    'crescimentoD28',
    'mediaAvaliacao28d',
    'mediaAvaliacao56d',
    'mediaAvaliacao84d',
]

drop_step = selection.DropFeatures(features_to_drop=features_drop)


In [0]:
from sklearn import pipeline

meu_pipeline = pipeline.Pipeline(steps=[
    ('Imput Media', imputer_mean),
    ('Impute Zeros', imputer_zero),
    ("Impute Uns", imputer_one),
    ("Impute Max", imputer_max),
    ("Impute Categorias Missing", imputer_flag_missing),
    ("Remocao", drop_step),
])

meu_pipeline.fit(X_train, y_train)

In [0]:
X_train_transform = meu_pipeline.transform(X_train)

In [0]:
X_train_transform

---

Some Tests

In [0]:
import pandas as pd

pd.set_option('display.max_rows', 500)

df_analysis = X_train_transform.copy()
df_analysis[target] = y_train

df_analysis.select_dtypes(exclude='object').groupby(target).median().T

In [0]:
cat_vars_analysis = df_analysis.select_dtypes(include='object').columns.tolist()

for n in cat_vars_analysis:
        print(f"Feature: {n}")
        print(df_analysis.groupby(n)[target].mean().sort_values(ascending=False))
        print('\n', "-" * 10, end='\n\n')